# Verknüpfung von Transaction lines und Products

Bei 192 Lines fehlt die product_id.

17 davon gehören zu einer gelabelten Transaktion, in allen Fällen ist das Label "FRAUD".

In [4]:
import polars as pl
from pathlib import Path

data_dir = Path("../data")

transactions = pl.scan_parquet(data_dir / "transactions_train_3.parquet")
lines = pl.scan_parquet(data_dir / "transaction_lines_train_3.parquet")
products = pl.scan_csv(data_dir / "products.csv")

prüfe, dass jedes Produkt auch eine base_product_id hat

In [5]:
products.select(["id", "base_product_id"]).describe().filter(pl.col("statistic").is_in(["count", "null_count"]))

statistic,id,base_product_id
str,str,str
"""count""","""8120""","""8120"""
"""null_count""","""0""","""0"""


In [6]:
# number of lines in the lines table
n_rows_lines = lines.select(pl.len()).collect().item()
print(f"{'Number of lines in lines:':<40} {n_rows_lines}")

# join lines and products
lines_products = lines.join(
    products,
    left_on="product_id",
    right_on="id",
    how="inner",
    suffix="_product",
)

# number of lines in the lines_products table
n_rows_lines_products = lines_products.select(pl.len()).collect().item()
print(f"{'Number of rows in joined lines_products:':<40} {n_rows_lines_products}")

# difference in number of rows
missing_rows = n_rows_lines - n_rows_lines_products
print(f"{'Difference in number of rows:':<40} {missing_rows}")


Number of lines in lines:                15793671
Number of rows in joined lines_products: 15793479
Difference in number of rows:            192


Grund der Differenz: Lines mit fehlenden product_id

In [7]:
lines.select("product_id").describe().filter(pl.col("statistic").is_in(["count", "null_count"]))

statistic,product_id
str,str
"""count""","""15793479"""
"""null_count""","""192"""


### Join transactions und lines und prüfe die Zeilen mit fehlender product_id

- Lines mit fehlender product_id haben in der Spalte "was_voided" den Wert true
- Kamera hat die Produkte nie erkannt (mit hoher Sicherheit)
- zeitlich sind die Transaktionen über die zwei Jahre verteilt

In [8]:
joined = (
    transactions.join(
        lines,
        left_on="id",
        right_on="transaction_id",
        suffix="__lines",
        how="left",
    )
    .filter(pl.col("id__lines").is_not_null())
    .filter(pl.col("product_id").is_null())
    .select(
        [
            pl.col("id").alias("transaction_id"),
            "n_lines",
            "id__lines",
            "product_id",
            "transaction_start",
            "was_voided",
            "camera_product_similar",
            "camera_certainty",
            "label",
        ],
    )
)

joined.describe()

statistic,transaction_id,n_lines,id__lines,product_id,transaction_start,was_voided,camera_product_similar,camera_certainty,label
str,str,f64,f64,str,str,f64,f64,f64,str
"""count""","""192""",192.0,192.0,"""0""","""192""",192.0,192.0,192.0,"""192"""
"""null_count""","""0""",0.0,0.0,"""192""","""0""",0.0,0.0,0.0,"""0"""
"""mean""",null,12.208333,5.020833,null,"""2022-10-09 15:48:00.780292""",1.0,0.0,0.961604,null
"""std""",null,12.643246,4.940439,null,null,null,null,0.055233,null
"""min""","""013253c0-db47-424e-8e31-eb84cc…",0.0,1.0,null,"""2022-02-02 10:08:39""",1.0,0.0,0.718538,"""FRAUD"""
"""25%""",null,3.0,2.0,null,"""2022-04-16 20:13:27""",null,null,0.93736,null
"""50%""",null,7.0,4.0,null,"""2022-08-05 19:27:57""",null,null,0.999941,null
"""75%""",null,16.0,6.0,null,"""2023-04-01 17:11:48""",null,null,1.0,null
"""max""","""fe186f14-5161-472e-958e-456b99…",65.0,41.0,null,"""2023-12-29 20:25:42""",1.0,0.0,1.0,"""UNKNOWN"""


In [9]:
n_missing_product_id = joined.select(pl.len()).collect().item()
print(
    f"Number of rows without product_id in joined transactions and lines: {n_missing_product_id}"
)

n_unique_transactions = (
    joined.select("transaction_id").unique().select(pl.len()).collect().item()
)
print(
    f"Number of unique transactions without product_id in joined transactions and lines: {n_unique_transactions}"
)

Number of rows without product_id in joined transactions and lines: 192
Number of unique transactions without product_id in joined transactions and lines: 182


In [11]:
vc = (
    joined.group_by("transaction_id", "n_lines", "label")
    .agg(pl.len().alias("n_missing_product_ids"))
    .sort("n_missing_product_ids")
    .collect()
)
print("Number of missing product_ids per transaction:")
vc


Number of missing product_ids per transaction:


transaction_id,n_lines,label,n_missing_product_ids
str,i64,str,u32
"""2298bdf8-c903-4ef6-b4f8-8f576e…",12,"""FRAUD""",1
"""3dd5b29c-064c-4fd1-af04-ef5a5b…",3,"""UNKNOWN""",1
"""fdfe7ebc-c991-4113-834d-74d645…",5,"""UNKNOWN""",1
"""ea492d1d-ea02-4f02-8642-170fb7…",6,"""UNKNOWN""",1
"""db19323a-cefa-4beb-9eb1-603011…",7,"""UNKNOWN""",1
…,…,…,…
"""f8c5318e-cf0f-4ad8-b7a5-60d3c1…",18,"""UNKNOWN""",2
"""2a3cb6e4-1b55-48ab-b0eb-a9d5cc…",13,"""UNKNOWN""",2
"""bd8b1528-321f-4851-96c7-229502…",14,"""UNKNOWN""",2


In [19]:
vc.filter(pl.col("label") == "FRAUD")

transaction_id,label,n_missing_product_ids
str,str,u32
"""e83ea717-a830-4b61-a050-272ae6…","""FRAUD""",1
"""6696b83d-7553-4725-8831-6f1a24…","""FRAUD""",1
"""1408e4f3-16cd-4125-920d-55cb97…","""FRAUD""",1
"""ec46aae0-b102-47d0-a4e6-019209…","""FRAUD""",1
"""83fc9067-6917-4f65-9f0a-766955…","""FRAUD""",1
…,…,…
"""09614115-d033-4f2e-80b4-fc8460…","""FRAUD""",1
"""968cd3e4-4799-4e80-9741-05b934…","""FRAUD""",1
"""2298bdf8-c903-4ef6-b4f8-8f576e…","""FRAUD""",1


In 17 Fällen gehört die Zeile zu einem Betrugsfall. 

In [20]:
vc = joined.group_by("label").agg(pl.len().alias("n_missing_product_ids")).collect()
print("Number of missing product_ids per label:")
vc

Number of missing product_ids per label:


label,n_missing_product_ids
str,u32
"""FRAUD""",17
"""UNKNOWN""",175
